In [10]:
import pandas as pd
import cvxpy as cvx
import numpy as np

In [27]:
pd.DataFrame([[1,2,3],[23,34,35]])

,0,1,2
0,1,2,3
1,23,34,35


# 准备数据

In [22]:
stocks = ['000001', '000002', '000003', '000004', '000005', '000006','000007']
factors = ['A','B','C']

In [23]:
loadings = pd.DataFrame(data=np.arange(1,22).reshape(3,7).T, index=stocks, columns=factors)

In [26]:
loadings.describe()

,A,B,C
count,7.000000,7.000000,7.000000
mean,4.000000,11.000000,18.000000
std,2.160247,2.160247,2.160247
min,1.000000,8.000000,15.000000
25%,2.500000,9.500000,16.500000
50%,4.000000,11.000000,18.000000
75%,5.500000,12.500000,19.500000
max,7.000000,14.000000,21.000000


In [4]:
loadings /= loadings.sum()

In [5]:
loadings

,A,B,C
000001,0.019608,0.035088,0.047619
000002,0.078431,0.087719,0.095238
000003,0.137255,0.140351,0.142857
000004,0.196078,0.192982,0.190476
000005,0.254902,0.245614,0.238095
000006,0.313725,0.298246,0.285714


In [7]:
min_exposures = pd.Series([-0.3,-0.1,-0.6], index=factors)
max_exposures = pd.Series([0.2,0.5,0.3], index=factors)

In [8]:
min_exposures

A   -0.3
B   -0.1
C   -0.6
dtype: float64

In [9]:
max_exposures

A    0.2
B    0.5
C    0.3
dtype: float64

# 模拟

In [11]:
n = len(stocks)

In [12]:
w_long = cvx.Variable(n,nonneg=True)
w_short = cvx.Variable(n,nonpos=True)

In [15]:
long_df_w = pd.Series([w_long[i] for i in range(n)], index = stocks)
short_df_w = pd.Series([w_short[i] for i in range(n)], index = stocks)

In [44]:
df_w = long_df_w - short_df_w

In [16]:
returns = pd.Series([0.1,0.2,0.3,0.4,0.4,-0.2], index=stocks)

In [27]:
obj = cvx.Maximize(cvx.sum((returns * long_df_w + returns * short_df_w).tolist()))

In [32]:
min_limit = pd.Series([0.1,0.3], index=['000002','000004'])
max_limit = pd.Series([0.1,0.3], index=['000001','000003'])

In [31]:
default_min = 0.1
default_max = 0.5

In [34]:
min_limit = min_limit.loc[stocks]
max_limit = max_limit.loc[stocks]

In [36]:
min_limit.fillna(default_min, inplace=True)
max_limit.fillna(default_max, inplace=True)

In [37]:
min_limit

000001    0.1
000002    0.1
000003    0.1
000004    0.3
000005    0.1
000006    0.1
dtype: float64

In [38]:
min_limit_cons = [long_df_w[idx] >= min_limit[idx] for idx in long_df_w.index]

In [39]:
max_limit_cons = [long_df_w[idx] >= max_limit[idx] for idx in long_df_w.index]

In [41]:
short_cons = [short_df_w[idx] >= 0 for idx in short_df_w.index]

In [50]:
long_cons = [cvx.sum(long_df_w.tolist()) <= 1.0]

In [51]:
prob = cvx.Problem(obj, min_limit_cons + max_limit_cons + short_cons + long_cons)

In [52]:
prob.solve()

-inf

In [49]:
prob.is_dcp()

True

In [76]:
x = cvx.Variable(10, nonneg=True)

In [77]:
obj = cvx.Maximize(cvx.sum(x))

In [79]:
cons = [x <= 10,
        x[0] <= 1,
        x[3] == 3]

In [80]:
prob = cvx.Problem(obj, cons)

In [81]:
prob.solve()

84.0

In [82]:
x.value

array([ 1., 10., 10.,  3., 10., 10., 10., 10., 10., 10.])